# Auto Polling
    - The original persona generation worked, but the personas were too generic and didnt give a model context to 
        make accurate predictions of the beliefs of a person
    - Now making a new pipeline that will create personas in different stages to 
        

In [1]:
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures
import os
import string
import random
import numpy as np
import pickle

In [2]:
def create_directory_if_not_exists(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# Load First and Last Names

In [3]:
eths = ["White", "American Indian or Alaska Native", "Black or African American", "Asian", "Hispanic or Latino"]
genders = ['Men', 'Women']

dems = []
for e in eths:
    for g in genders:
        s = f"{e} {g}"
        dems.append(s)
        
first_names = {}
for d in dems:
    # Open the file in read binary mode
    with open(f"Names/First_Names/{d}.pkl", "rb") as file:
        # Use pickle to load the list object from the file
        my_list = pickle.load(file)
        first_names[d] = my_list

last_names = {}
for e in eths:
    # Open the file in read binary mode
    with open(f"Names/Last_Names/{e}.pkl", "rb") as file:
        # Use pickle to load the list object from the file
        my_list = pickle.load(file)
        last_names[e] = my_list

# Distribute Geographic Location

In [4]:
def distribute_geographic_location():
    random_number = random.random()
    
    if random_number < 0.44:
        return "Rural Montana, United States"
    else:
        return "Urban Montana, United States"

# Distribute Gender

In [5]:
def distribute_gender():
    random_number = random.random()
    
    if random_number < 0.49:
        return "Female"
    else:
        return "Male"

# Distribute Ethnicity

In [6]:
def distribute_ethnicity():
    random_number = random.random()
    
    if random_number < 0.88:
        return "White"
    elif random_number < 0.94:
        return "American Indian or Alaska Native"
    elif random_number < 0.95:
        return "Black or African American"
    elif random_number < 0.96:
        return "Asian"
    else:
        return "Hispanic or Latino"

# Distribute Age and Income

In [7]:
def distribute_age_and_income():
    random_number = random.random()
    
    if random_number < 0.13:
        age = random.randint(18, 25)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 32413  # Mean of the distribution
        std_dev = 10000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    elif random_number < 0.44:
        age = random.randint(25, 44)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 69192  # Mean of the distribution
        std_dev = 20000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    elif random_number < 0.75:
        age = random.randint(45, 64)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 71064  # Mean of the distribution
        std_dev = 25000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    else:
        age = random.randint(65, 87)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 47015  # Mean of the distribution
        std_dev = 15000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income

# Distribute the Education Levels

In [8]:
def distribute_education_level():
    random_number = random.random()
    
    if random_number < 0.60:
        return "High school graduate"
    elif random_number < 0.94:
        return "Bachelor's Degree or Higher"
    else:
        return "Less than a high school education"

# Distribute Politcal Affiliation
    - Reps that say they are conservative at Con
    - Reps say moderate, right leaning
    - Anyone that contradicts or says dont know is moderate
    - Dems that say moderate, left leaning
    - Dems that say liberal, liberal

In [9]:
def distribute_party():
    random_number = random.random()
    
    if random_number < 0.42:
        return "Conservative"
    elif random_number < 0.56:
        return "Right Leaning"
    elif random_number < 0.66:
        return "Moderate"
    elif random_number < 0.81:
        return "Left Leaning"
    else:
        return "Liberal"

# Distribute Veteran Status

In [10]:
def distribute_veteran():
    random_number = random.random()
    
    if random_number < 0.9:
        return "Not a Veteran"
    else:
        return "Is a Veteran"

# Generate Names

In [11]:
def generate_name(p):
    if p['Gender'] == 'Male':
        gen = 'Men'
    else:
        gen = 'Women'
    f_names = first_names[f"{p['Ethnicity']} {gen}"]
    
    first = f_names[random.randint(0,len(f_names)-1)].strip()
    
    l_names = last_names[f"{p['Ethnicity']}"]
    last = l_names[random.randint(0,len(l_names)-1)].strip()
    
    return f"{first} {last}"

In [12]:
def make_persona():
    p = {}
    # Distribute Age and Income
    age, income = distribute_age_and_income()
    p['Age'] = str(age)
    
    # Distribute Gender
    p['Gender'] = distribute_gender()
    p['Income'] = str(income)
    
    # Distribute Ethnicity
    p['Ethnicity'] = distribute_ethnicity()
    
    # Distribute Education Level
    p['Education Level'] = distribute_education_level()
    
    # Distribute Political Affiliation
    p['Political Affiliation'] = distribute_party()
    
    # Distribute Geographic Location
    p['Geographic Location'] = distribute_geographic_location()
    
    # Distribute Veteran Status
    p['Veteran Status'] = distribute_veteran()
    
    p['Name'] = generate_name(p)
    
    return p

In [13]:
poll_size = 10
pp = []
for i in range(poll_size):
    pp.append(make_persona())

In [14]:
def gen_person_occupation(p):
    occ_gen_role = GPT.open_file("Prompts/OccupationGen/occ_gen_role.txt")
    prompt = GPT.open_file("Prompts/OccupationGen/occ_gen_prompt.txt").replace('<<PERSON>>', str(p))
    occ, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, role=occ_gen_role)
    
    p['Occupation'] = occ
    
    return p

In [15]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_occ = executor.map(gen_person_occupation, pp)

In [16]:
# for p in pp_occ:
#     print(p, '\n')

In [17]:
def gen_person_media(p):
    media_gen_role = GPT.open_file("Prompts/MediaGen/media_gen_role.txt")
    prompt = GPT.open_file("Prompts/MediaGen/media_gen_in.txt").replace('<<PERSON>>', str(p))
    media, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, role=media_gen_role)
    
    p['Media Consumption'] = media
    
    return p

In [18]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_media = executor.map(gen_person_media, pp_occ)

In [20]:
ppp = list(pp_media)

In [22]:
for p in ppp:
    print(p, '\n')

{'Age': '22', 'Gender': 'Male', 'Income': '48500', 'Ethnicity': 'White', 'Education Level': 'High school graduate', 'Political Affiliation': 'Conservative', 'Geographic Location': 'Urban Montana, United States', 'Veteran Status': 'Not a Veteran', 'Name': 'Louis Watson', 'Occupation': 'Construction worker.', 'Media Consumption': 'Based on the demographic information provided, Louis Watson, a 22-year-old male, high school graduate, and construction worker from urban Montana, United States, with a conservative political affiliation, is likely to trust and consume the following media outlets regularly:\n\n1. Fox News - Louis is likely to trust and consume Fox News regularly as it is known for its conservative bias and is a popular news source among conservatives.\n\n2. The Daily Wire - This news outlet is known for its conservative views and is likely to be trusted and consumed by Louis.\n\n3. The Blaze - This news outlet is also known for its conservative views and is likely to be trusted

In [31]:
def gen_person_voting_history(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_voting_history.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, role=voting_gen_role)
    
    p['Voting History'] = vh
    
    return p

In [32]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_vote = executor.map(gen_person_voting_history, ppp)

In [33]:
pppp = list(pp_vote)

In [34]:
pppp[0]

{'Age': '22',
 'Gender': 'Male',
 'Income': '48500',
 'Ethnicity': 'White',
 'Education Level': 'High school graduate',
 'Political Affiliation': 'Conservative',
 'Geographic Location': 'Urban Montana, United States',
 'Veteran Status': 'Not a Veteran',
 'Name': 'Louis Watson',
 'Occupation': 'Construction worker.',
 'Media Consumption': 'Based on the demographic information provided, Louis Watson, a 22-year-old male, high school graduate, and construction worker from urban Montana, United States, with a conservative political affiliation, is likely to trust and consume the following media outlets regularly:\n\n1. Fox News - Louis is likely to trust and consume Fox News regularly as it is known for its conservative bias and is a popular news source among conservatives.\n\n2. The Daily Wire - This news outlet is known for its conservative views and is likely to be trusted and consumed by Louis.\n\n3. The Blaze - This news outlet is also known for its conservative views and is likely to 

In [39]:
def gen_person_group_membership(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_group_membership.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, role=voting_gen_role)
    
    p['Group Membership'] = vh
    
    return p

In [40]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_mem = executor.map(gen_person_group_membership, pppp)

In [41]:
ppppp = list(pp_mem)

In [42]:
ppppp[0]

{'Age': '22',
 'Gender': 'Male',
 'Income': '48500',
 'Ethnicity': 'White',
 'Education Level': 'High school graduate',
 'Political Affiliation': 'Conservative',
 'Geographic Location': 'Urban Montana, United States',
 'Veteran Status': 'Not a Veteran',
 'Name': 'Louis Watson',
 'Occupation': 'Construction worker.',
 'Media Consumption': 'Based on the demographic information provided, Louis Watson, a 22-year-old male, high school graduate, and construction worker from urban Montana, United States, with a conservative political affiliation, is likely to trust and consume the following media outlets regularly:\n\n1. Fox News - Louis is likely to trust and consume Fox News regularly as it is known for its conservative bias and is a popular news source among conservatives.\n\n2. The Daily Wire - This news outlet is known for its conservative views and is likely to be trusted and consumed by Louis.\n\n3. The Blaze - This news outlet is also known for its conservative views and is likely to 

In [43]:
def gen_person_group_membership(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_influential_figures.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, role=voting_gen_role)
    
    p['Influential Figures'] = vh
    
    return p

In [44]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_if = executor.map(gen_person_group_membership, ppppp)

In [45]:
pppppp = list(pp_if)

In [46]:
pppppp[0]

{'Age': '22',
 'Gender': 'Male',
 'Income': '48500',
 'Ethnicity': 'White',
 'Education Level': 'High school graduate',
 'Political Affiliation': 'Conservative',
 'Geographic Location': 'Urban Montana, United States',
 'Veteran Status': 'Not a Veteran',
 'Name': 'Louis Watson',
 'Occupation': 'Construction worker.',
 'Media Consumption': 'Based on the demographic information provided, Louis Watson, a 22-year-old male, high school graduate, and construction worker from urban Montana, United States, with a conservative political affiliation, is likely to trust and consume the following media outlets regularly:\n\n1. Fox News - Louis is likely to trust and consume Fox News regularly as it is known for its conservative bias and is a popular news source among conservatives.\n\n2. The Daily Wire - This news outlet is known for its conservative views and is likely to be trusted and consumed by Louis.\n\n3. The Blaze - This news outlet is also known for its conservative views and is likely to 

In [49]:
def gen_person_personal_anecdotes(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/personal_anecdotes.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, role=voting_gen_role)
    
    p['Personal Anecdotes'] = vh
    
    return p

In [50]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_anec = executor.map(gen_person_personal_anecdotes, pppppp)

In [51]:
ppppppp = list(pp_anec)
ppppppp[0]

{'Age': '22',
 'Gender': 'Male',
 'Income': '48500',
 'Ethnicity': 'White',
 'Education Level': 'High school graduate',
 'Political Affiliation': 'Conservative',
 'Geographic Location': 'Urban Montana, United States',
 'Veteran Status': 'Not a Veteran',
 'Name': 'Louis Watson',
 'Occupation': 'Construction worker.',
 'Media Consumption': 'Based on the demographic information provided, Louis Watson, a 22-year-old male, high school graduate, and construction worker from urban Montana, United States, with a conservative political affiliation, is likely to trust and consume the following media outlets regularly:\n\n1. Fox News - Louis is likely to trust and consume Fox News regularly as it is known for its conservative bias and is a popular news source among conservatives.\n\n2. The Daily Wire - This news outlet is known for its conservative views and is likely to be trusted and consumed by Louis.\n\n3. The Blaze - This news outlet is also known for its conservative views and is likely to 

In [ ]:
def gen_person_personal_anecdotes(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_future_aspirations_and_concerns.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, role=voting_gen_role)
    
    p['Future Aspirations/Concerns'] = vh
    
    return p

In [ ]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_anec = executor.map(gen_person_personal_anecdotes, ppppppp)

In [28]:
def dict_to_paragraph(persona, residence):
    priorities = ', '.join(persona["Top Priorities"])
    try:
        pain_points = ', '.join(persona["Pain points"])
    except:
        pain_points = ', '.join(persona["Pain Points"])
    
    return (f"Your name is {persona['Name']}, you are a {persona['Age']} year old {persona['Ethnicity']} {persona['Gender']}. "
            f"You live in {residence} and your annual income is ${persona['Income']}. "
            f"Your marital status is {persona['Marital Status']}, and your highest level of education is {persona['Education Level']}. "
            f"You work as a {persona['Occupation']}. You are described as '{persona['Description']}'. "
            f"Your top priorities are {priorities}. You are facing some challenges, including {pain_points}.")
